# Test Automation

Runs Test_ML_Models_A in a loop

In [14]:
import numpy as np
import pandas as pd
import script_utils as mL
import pickle
import os
import time
import gc

NDBC = mL.NDBC_lib
ERA5 = mL.ERA5_lib
Models = mL.Models
DP = mL.DataProcessor
Experiment = mL.Experiment

In [15]:
def run_MLM_test(MODEL_NAME, ALPHA, filename, report_description,
               DATAFILE = "dataset_GOM_1_A_A.pickle", STATIONARY_SHIFT = 1, N_TEST_HOURS=24):

    #Read data from file
    data_directory = os.path.join(os.getcwd(), f'data/datasets/type_A')
    with open(f'data/datasets/publication/{DATAFILE}', 'rb') as f:
        dataset = pickle.load(f)
    data = dataset["data"]

    #Preprocessing
    data_stationary = DP.data_to_stationary(data, n = STATIONARY_SHIFT)
    data_supervised = DP.data_to_supervised(data_stationary, n_in=3)

    train_X, train_y, test_X, test_y = DP.train_test_split(data_supervised, N_TEST_HOURS)

    #Training
    start_time = time.time()
    model = Models.get_model(MODEL_NAME, train_X, train_y, ALPHA)
    TRAINING_TIME = time.time() - start_time

    #One-Shot-Forecasting
    model.predict(train_X, batch_size=1)
    yhat = model.predict(test_X)

    #Create Evaluation Dataframes
    output_cols = data.columns.tolist()
    yhat_df = pd.DataFrame(yhat, columns=[name + "_pred" for name in output_cols])
    yhat_df.set_index(data.tail(len(yhat)).index, inplace=True)

    evaluation_1 = data.tail(len(yhat)+1).copy()  #+1 since i need that value for de-differencing

    #De-Differenciating
    for col in evaluation_1.columns:
        evaluation_1[f"{col}_pred"]= evaluation_1[col].shift(STATIONARY_SHIFT) + yhat_df[f"{col}_pred"]

    evaluation_1 = evaluation_1.iloc[STATIONARY_SHIFT:]  # remove first n entries since there is no delta value for them

    # Correct wind direction (modulo 360)
    wdir_columns = [col for col in evaluation_1.columns if col.startswith("WDIR")]
    evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360

    #CREATE REPORT
    # Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
        name=filename,
        description=report_description,

        stations = dataset["stations"],
        years = dataset["years"],
        nan_threshold=dataset["nan_threshold"],
        features=dataset["features"],
        era5=dataset["add_era5"],

        stationary_shift=STATIONARY_SHIFT,

        n_test_hours=N_TEST_HOURS,

        #stationary=STATIONARY,
        scaler= None, # SCALER,

        model_name = MODEL_NAME,
        model_summary=model_summary,
        training_time = TRAINING_TIME,

        one_shot_forecast = evaluation_1,
        recursive_forecast = None   # evaluation_2
    )


    # open a file for writing in binary mode
    filepath = f'data/reports/{report.name}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print(f"File successfully saved:{filepath}")

In [16]:
def run_SSUM_test(MODEL_NAME, ALPHA, filename, report_description,
               DATAFILE = "GOM_buoys_since2002_SSUM.pickle", STATIONARY_SHIFT = 1, N_TEST_HOURS=24):   #nan_threshold=0.66 (original): type_B/dataset_GOM_1_B_B.pickle    #current version is modification from september 5th - it uses the dataset SSUM/GOM_buoys_since2002_SSUM.pickle with nan rate of 0.33

    #Read data from file
    with open(f'data/datasets/SSUM/{DATAFILE}', 'rb') as f:
        # load the object from the file using pickle.load()
        dataset = pickle.load(f)

    train = dataset["data_train"]
    test = dataset["data_test"]

    #Preprocessing
    train_stationary = DP.data_to_stationary(train, n = STATIONARY_SHIFT)
    test_stationary = DP.data_to_stationary(test, n = STATIONARY_SHIFT)

    #Data is already supervised!

    train_X, train_y, _, _ = DP.train_test_split(train_stationary, -len(train_stationary))
    _, _, test_X, test_y = DP.train_test_split(test_stationary, len(test_stationary))

    #Training
    start_time = time.time()
    model = Models.get_model(MODEL_NAME, train_X, train_y, ALPHA)
    TRAINING_TIME = time.time() - start_time

    #One-Shot-Forecasting
    model.predict(train_X, batch_size=1)
    yhat = model.predict(test_X)

    #Create Evaluation Dataframes
    output_cols  = test.loc[:, ~test.columns.str.contains('t-')].columns

    yhat_df = pd.DataFrame(yhat)
    yhat_df.columns = [name + "_pred" for name in output_cols]
    yhat_df.set_index(test.tail(len(yhat)).index, inplace=True)
    evaluation_1 = test.loc[:, ~test.columns.str.contains('t-')]    #ground truth: just y, without X

    #De-Differenciating
    for col in evaluation_1.columns:
        shifted = evaluation_1[col].shift(STATIONARY_SHIFT)
        evaluation_1[f"{col}_pred"]= shifted + yhat_df[f"{col}_pred"]

    evaluation_1 = evaluation_1.iloc[STATIONARY_SHIFT:]  # remove first n entries since there is no delta value for them

    # Correct wind direction (modulo 360)
    wdir_columns = [col for col in evaluation_1.columns if col.startswith("WDIR")]
    evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360

    #CREATE REPORT
    #Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
            name=filename,
            description=report_description,

            stations = dataset["files"],
            years = ["not available"],
            nan_threshold=dataset["nan_threshold"],
            features=dataset["features"],
            era5=dataset["add_era5"],

            stationary_shift=STATIONARY_SHIFT,
            # lag=1,
            n_test_hours=dataset["num_test_hours"],

            #stationary=STATIONARY,
            scaler= None, # SCALER,

            model_name = MODEL_NAME,
            model_summary=model_summary,
            training_time = TRAINING_TIME,

            one_shot_forecast = evaluation_1,
            recursive_forecast = None
    )

    # open a file for writing in binary mode
    filepath = f'data/reports/{filename}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print("File successfully saved:")
        print(filepath)


    #Release Data Variables to reduce RAM usage
    del train
    del test
    del train_stationary
    del test_stationary
    del train_X
    del train_y
    del test_X
    del test_y
    del model
    del yhat
    del stringlist
    del model_summary
    del filepath
    del report
    gc.collect()

# Execute All Tests:

In [ ]:
datafile ="GOM_stage_2.pickle"
approach = "MLM"   # "MLM" ... Multi Location Modelling, "SSUD" ... Station Specific Unified Dataset
model_names = ["transformer", "LSTM_2", "CNN_2"] # ["GRU", "CNN", "TCN"]
alpha_values = np.arange(0.0, 1.1, 0.1) #from, to, increment
report_description = "Executed with automated script. With random seed. used shrinked dataset stage 1"

print(alpha_values)

In [13]:
if approach == "MLM":
    for model in model_names:
        for alpha in alpha_values:
            print(f"Started {model} - {alpha}")
            filename = f'report_A_{model}_{format(alpha, ".1f")}'
            run_MLM_test(model,alpha,filename,report_description, DATAFILE = datafile)

elif approach == "SSUD":
    for model in model_names:
        for alpha in alpha_values:
            filename = f'report_B_{model}_{format(alpha, ".1f")}'
            run_SSUM_test(model,alpha,filename,report_description)
            print(filename)

Started transformer - 0.0
Epoch 1/100


KeyboardInterrupt: 

In [ ]:
print("DONE")

# Execute specific tests:

In [ ]:
datafile ="GOM_stage_1.pickle"

experiment = [["LSTM_2",        0.7,    "report_A_LSTM_2_0.7"],
             ["CNN_2",          0.2,    "report_A_CNN_2_0.2"],
             ["transformer",    0.2,    "report_A_transformer_0.2"],
            ]

report_description = "Executed selected tests with stage_2 dataset and random seed!"

for e in experiment:
    model = e[0]
    alpha = e[1]
    filename = e[2]

    run_MLM_test(model,alpha,filename,report_description, DATAFILE = datafile)

Epoch 1/100
2589/2589 [==============================] - 22s 8ms/step - loss: 1.9524 - val_loss: 3.8861
Epoch 2/100
2589/2589 [==============================] - 27s 10ms/step - loss: 1.9429 - val_loss: 3.8564
Epoch 3/100
2589/2589 [==============================] - 17s 7ms/step - loss: 1.9372 - val_loss: 3.8416
Epoch 4/100
2589/2589 [==============================] - 17s 7ms/step - loss: 1.9315 - val_loss: 3.8273
Epoch 5/100
2589/2589 [==============================] - 17s 7ms/step - loss: 1.9271 - val_loss: 3.8149
Epoch 6/100
2589/2589 [==============================] - 17s 7ms/step - loss: 1.9170 - val_loss: 3.7964
Epoch 7/100
2589/2589 [==============================] - 19s 7ms/step - loss: 1.9087 - val_loss: 3.7893
Epoch 8/100
2589/2589 [==============================] - 17s 7ms/step - loss: 1.9033 - val_loss: 3.7846
Epoch 9/100
2589/2589 [==============================] - 17s 6ms/step - loss: 1.8987 - val_loss: 3.7833
Epoch 10/100
2589/2589 [==============================] - 18s 7